Instalações de bibliotecas necessárias:

OBS: A numpy possui um erro na biblioteca boruta_py, com isso é necessário abrir o arquivo borutapy e retirar todos os np.int, np.float, np.bool e converter para int, float e bool, sucessivamente.

In [6]:
'''!pip install pandas
!pip install sklearn
!pip install boruta
!pip install pgmpy
!pip install networkx
!pip install matplotlib'''

'!pip install pandas\n!pip install sklearn\n!pip install boruta\n!pip install pgmpy\n!pip install networkx\n!pip install matplotlib'

In [7]:
'''!pip install --upgrade --force-reinstall pgmpy'''


'!pip install --upgrade --force-reinstall pgmpy'

In [8]:
#Método para realizar o Boruta, e ao final, montar um DataFrame com a ordem selecionada pelo boruta

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.preprocessing import LabelEncoder

def boruta_feature_order(data_path, target_column):
    # Carregar os dados do CSV
    D = pd.read_csv(data_path)
    variables = D.columns
    print(f'Lista de variáveis: {variables}')

    # Print para verificar os dados antes do processamento
    print("Dados antes do processamento:")
    print(D)
    tipo_antes = (type(D[target_column][0]))
    print(f'Tipo dos dados antes do processamento: {tipo_antes}')
    

    # Mapear os valores nominais para números inteiros únicos
    D_encoded = D.apply(LabelEncoder().fit_transform)
    # Print para verificar os dados após o processamento
    print("Dados após o processamento:")
    print(D_encoded)
    tipo_depois = (type(D_encoded[target_column][0]))
    print(f'Tipo dos dados após o processamento (inteiros): {tipo_depois}')

    # Separar os dados em características (X) e alvo (y)
    X = D_encoded.drop(columns=[target_column])
    y = D_encoded[target_column]

    # Inicializar um classificador Random Forest
    rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

    # Inicializar o Boruta
    boruta_selector = BorutaPy(rf, n_estimators='auto', verbose=2)

    # Ajustar o Boruta aos dados
    boruta_selector.fit(X.values, y.values)
    print("Boruta inicializado com sucesso")

    # Obter as características selecionadas
    print("Variáveis na ordem inicial")
    print(variables)
    
    selected_features = X.columns[boruta_selector.support_]
    print("Features selecionadas")
    print(selected_features)
    # Obter as características não selecionadas
    unselected_features = X.columns[~boruta_selector.support_]
    print("Features não selecionadas")
    print(unselected_features)

    # Combinar características selecionadas e não selecionadas, ordenando-as pela ordem de ranking do Boruta
    all_features = [target_column] + list(selected_features) + list(unselected_features)
    print("Todas as features, incluindo a target: ")
    print(all_features)

    # Reorganizar o DataFrame original de acordo com a ordem das características selecionadas pelo Boruta
    df_reordered = D[all_features]
    
    # Salvar o DataFrame reordenado no formato CSV
    df_reordered.to_csv("data_reordered.csv", index=False)

    print("DataFrame reordenado salvo como data_reordered.csv")
    print("DataFrame criado na ordem das features selecionadas pelo Boruta")
    print(df_reordered)
    
    return all_features

# Caminho para o arquivo CSV e o nome da coluna alvo
data_path = "barley.csv"
target_column = "aar_mod"

# Obter todas as características, incluindo a coluna alvo
all_features= boruta_feature_order(data_path, target_column)
print("Features em ordem")
print(all_features)

Lista de variáveis: Index(['aar_mod', 'aks_m2', 'aks_vgt', 'antplnt', 'bgbyg', 'dg25', 'dgv1059',
       'dgv5980', 'exptgens', 'forfrugt', 'frspdag', 'jordinf', 'jordn',
       'jordtype', 'keraks', 'komm', 'ksort', 'markgrm', 'mod_nmin',
       'nedbarea', 'ngodn', 'ngodnn', 'ngodnt', 'ngtilg', 'nmin', 'nopt',
       'nplac', 'nprot', 'ntilg', 'partigerm', 'pesticid', 'potnmin',
       'protein', 'rokap', 's2225', 's2528', 'saakern', 'saamng', 'saatid',
       'slt22', 'sort', 'sorttkv', 'spndx', 'srtprot', 'srtsize', 'tkv',
       'tkvs', 'udb'],
      dtype='object')
Dados antes do processamento:
     aar_mod     aks_m2    aks_vgt   antplnt     bgbyg        dg25 dgv1059  \
0        x25   x650_750      x_550     x_175  x4_5_5_0  x27_5_32_5  x41_50   
1        x10  x950_1150   x650_700  x375_425     x_3_0  x32_5_37_5  x61_70   
2       x_15   x550_650   x700_750  x375_425  x4_0_4_5  x37_5_47_5  x41_50   
3        x25   x650_750  x950_1050  x325_375     x_3_0      x_22_5  x51_60   
4 

In [30]:
import pandas as pd
from pgmpy.estimators import K2Score
from sklearn.preprocessing import LabelEncoder
from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch, K2Score, BayesianEstimator


def load_data(DATA_CSV):
    D = pd.read_csv('data_reordered.csv')
    #D = D.apply(LabelEncoder().fit_transform)

    V = D.columns
    N = len(D.index)
    V_CARD = {v: len(D[v].unique()) for v in V}


    print(f'ARQUIVO: {D}')
    print(f'VARIÁVEIS: {V}')
    print(f'NÚMERO DE AMOSTRAS: {N}')
    print(f'MAPEAMENTO DAS VARIÁVEIS COM NÚMEROS DE VALORES ÚNICOS: {V_CARD}')
    return D, V, N, V_CARD

#Método K2 mais correto, que retorna as CPDs

def calcular_k2(D):
    """
    Calcula a pontuação K2 para os dados fornecidos.

    Args:
    - D: DataFrame contendo os dados.

    Returns:
    - k2score: Objeto K2Score contendo a pontuação K2 calculada.
    """
    k2score = K2Score(D)
    return k2score

def estimar_modelo(D, scoring_method):
    """
    Estima a estrutura do modelo usando o método K2.

    Args:
    - D: DataFrame contendo os dados.
    - scoring_method: Objeto de pontuação a ser usado para estimar a estrutura.

    Returns:
    - best_model: Modelo BayesianModel estimado.
    """
    estimator_k2 = HillClimbSearch(D)
    best_model = estimator_k2.estimate(scoring_method=scoring_method, max_indegree= 4,max_iter= 100)
    return best_model


def tabular_cpd(best_model, D):
    """
    Estima as CPDs para o modelo usando o estimador bayesiano.

    Args:
    - best_model: Modelo BayesianModel estimado.
    - D: DataFrame contendo os dados.

    Returns:
    - cpds: Lista de CPDs estimadas.
    """
    bayesian_network = BayesianModel(best_model)
    estimator = BayesianEstimator(bayesian_network, D)
    cpds = []
    for node in bayesian_network.nodes():
        cpd = estimator.estimate_cpd(node)
        cpds.append(cpd)
    return cpds, bayesian_network

# Dados já reordenados pelo Boruta, e suas propriedades
D = pd.read_csv('data_reordered.csv')
print(D)

V = D.columns
N = len(D.index)

print(f'VARIÁVEIS: {V}')
print(f'NÚMERO DE AMOSTRAS: {N}')

# Calcula a pontuação K2
k2score = calcular_k2(D)
# Estima a estrutura do modelo com o K2
best_model = estimar_modelo(D, k2score)
print(f'Melhor modelo: {best_model}')

# Valor do Score gerado pelo K2
k2_score = k2score.score(best_model)
print(f'Valor do score K2: {k2_score}')

# Exibe a estrutura do modelo
structure = (best_model)
print(f'Estrutura da rede: {structure}')


# Estima as CPDs e passa a bayesian_netq
cpds, bayesian_network = tabular_cpd(best_model, D)
print(f'CPDs: {cpds}')
print(f'Bayesian Network: {bayesian_network}')
#Juntar o best_model com as cpds, para passar para o arquivoxml

     aar_mod mod_nmin    nmin     aks_m2    aks_vgt   antplnt     bgbyg  \
0        x25    x0_15   x0_15   x650_750      x_550     x_175  x4_5_5_0   
1        x10   x15_30  x30_45  x950_1150   x650_700  x375_425     x_3_0   
2       x_15   x45_60  x30_45   x550_650   x700_750  x375_425  x4_0_4_5   
3        x25    x0_15  x30_45   x650_750  x950_1050  x325_375     x_3_0   
4         x5   x15_30  x15_30   x650_750   x750_850  x375_425  x4_5_5_0   
...      ...      ...     ...        ...        ...       ...       ...   
9995    x_10   x30_45  x15_30      x_350     x_1050  x375_425  x4_5_5_0   
9996     x10    x0_15  x15_30  x950_1150   x700_750  x175_225  x4_5_5_0   
9997     x20    x0_15  x15_30     x_1150   x550_600  x325_375  x4_0_4_5   
9998     x25    x0_15  x30_45      x_350     x_1050  x375_425  x4_5_5_0   
9999     x20   x30_45  x45_60  x950_1150   x600_650  x175_225  x4_0_4_5   

            dg25 dgv1059 dgv5980  ...      saatid slt22      sort sorttkv  \
0     x27_5_32_5  x41_

100%|██████████| 100/100 [00:51<00:00,  1.96it/s]


Melhor modelo: DAG with 48 nodes and 100 edges


Valor do score K2: 12374532.085786061
Estrutura da rede: DAG with 48 nodes and 100 edges
CPDs: [<TabularCPD representing P(aar_mod:11 | mod_nmin:6, nmin:5, s2528:20, sort:67) at 0x16c289116a0>, <TabularCPD representing P(mod_nmin:6 | exptgens:6, ngodnt:9, s2528:20, sort:67) at 0x16c26dac6b0>, <TabularCPD representing P(nmin:5 | jordtype:9, nedbarea:3) at 0x16c26da86e0>, <TabularCPD representing P(aks_m2:8 | aks_vgt:9, s2528:20, sort:67, udb:9) at 0x16c26da8ad0>, <TabularCPD representing P(aks_vgt:9 | keraks:7, ksort:5, s2225:3, sort:67) at 0x16c26da0830>, <TabularCPD representing P(antplnt:7 | markgrm:10, saakern:7, sort:67, spndx:3) at 0x16c26de4860>, <TabularCPD representing P(bgbyg:6) at 0x16c26d8c2f0>, <TabularCPD representing P(dg25:7 | saatid:5) at 0x16c26d8c200>, <TabularCPD representing P(dgv1059:6 | rokap:7, saatid:5, sort:67, spndx:3) at 0x16c26d8c950>, <TabularCPD representing P(dgv5980:6 | bgbyg:6) at 0x16c26d742f0>, <TabularCPD representing P(exptgens:6 | nopt:6) at 0x16c2

In [34]:
from pgmpy.readwrite import XMLBIFWriter

# Inicialize o Modelo Bayesian
modelo_completo = BayesianModel(bayesian_network)  # Inicialize com as arestas do bayesian_network
print(f'Modelo: {modelo_completo}')

# Adicionando as CPDs ao Modelo
for cpd in cpds:
    modelo_completo.add_cpds(cpd)

# Verifique a consistência do modelo
assert modelo_completo.check_model()

# Verificar a Estrutura do Modelo
print("Estrutura do Modelo:")
print(modelo_completo.edges())

# Verificar as CPDs e imprimir na tela
'''print("\nCPDs do Modelo:")
for node in modelo_completo.nodes():
    print(f"CPD para o nó {node}:")
    print(modelo_completo.get_cpds(node))'''

# Verificar a Consistência do Modelo
print("\nVerificação de Consistência do Modelo:")
print(modelo_completo.check_model())

# Fazendo o formato XMLBIF
writer = XMLBIFWriter(modelo_completo)

# Especifique o nome do arquivo para salvar o modelo
nome_arquivo = "barley.xml"

# Escreva o modelo no arquivo .xmlbif
writer.write_xmlbif(nome_arquivo)  # Use write em vez de write_xmlbif


Modelo: BayesianModel with 48 nodes and 100 edges
Estrutura do Modelo:
[('aar_mod', 'potnmin'), ('aar_mod', 'sorttkv'), ('aar_mod', 'srtprot'), ('mod_nmin', 'potnmin'), ('mod_nmin', 'aar_mod'), ('nmin', 'aar_mod'), ('nmin', 'nprot'), ('aks_vgt', 'aks_m2'), ('aks_vgt', 'udb'), ('antplnt', 'saamng'), ('bgbyg', 'dgv5980'), ('dgv1059', 'keraks'), ('dgv1059', 'protein'), ('exptgens', 'mod_nmin'), ('exptgens', 'jordtype'), ('exptgens', 'rokap'), ('frspdag', 'saatid'), ('jordinf', 'partigerm'), ('jordinf', 'markgrm'), ('jordinf', 'frspdag'), ('jordn', 'ngodn'), ('jordn', 'ntilg'), ('jordn', 'potnmin'), ('jordn', 'ngodnn'), ('jordtype', 'nmin'), ('jordtype', 'forfrugt'), ('keraks', 'aks_vgt'), ('keraks', 's2225'), ('ksort', 'slt22'), ('ksort', 'aks_vgt'), ('markgrm', 'partigerm'), ('markgrm', 'antplnt'), ('markgrm', 'sort'), ('nedbarea', 'nmin'), ('ngodn', 'ngodnn'), ('ngodn', 'ngtilg'), ('ngodn', 'ngodnt'), ('ngodnn', 'ngtilg'), ('ngodnn', 'ngodnt'), ('ngodnt', 'mod_nmin'), ('ngtilg', 'ntilg'

KeyboardInterrupt: 